In [81]:
from tqdm.notebook import tqdm 
from collections import Counter 
import os 
import time 

# Лабораторна робота 3. Групи, порядок елемента в групі

_Примітка_: наведений код є лише одним з можливих шаблонів виконання. Можете писати по-своєму, але розділяйте свій код на функції, щоб їх можна було простіше перевіряти.

## Завдання 1. 

__Задайте групи $D_{10}$, $Z_{16}$, $A_5$, $SL(2, \mathbb{Z}_3)$*. Для кожної з них виведіть таблицю Келі, знайдіть порядок групи, перевірте чи група абелева. Перевірте чи будуть ізоморфними групи $SL(2, Z_3)$ та $D_{12}$.__

*: група матриць 2х2 з визначником 1 над полем $\mathbb{Z}_3$

In [9]:
# ваш код тут 

def describe(group, file):
    print('\n\n==================================', file=file)
    print('Group', group, 'which is isomorphic to ', group.structure_description(), file=file)
    print(ascii_art(group.cayley_table()), file=file)
    print('group is abelian:', group.is_abelian(), file=file)
    print('order of group:', group.order(), file=file)
    print('==================================\n\n', file=file)
    

folder = './lab03_assets/'
os.makedirs(folder, exist_ok=True)

with open(folder + 'task1_output.txt', 'w') as file: 
    for G in [
        DihedralGroup(10), 
        CyclicPermutationGroup(16), 
        AlternatingGroup(5), 
        SL(2, Integers(3)),
    ]: 
        describe(G, file)


In [11]:
!head lab03_assets/task1_output.txt
!echo "..."
!echo "..."
!echo "..."
!tail lab03_assets/task1_output.txt



Group Dihedral group of order 20 as a permutation group which is isomorphic to  D10
*  a b c d e f g h i j k l m n o p q r s t
 +----------------------------------------
a| a b c d e f g h i j k l m n o p q r s t
b| b a d c f e h g j i l k n m p o r q t s
c| c s a e d g f i h k j m l o n q p t b r
d| d t b f c h e j g l i n k p m r o s a q
...
...
...
t| x r u k n h t b j q a m w c g o v f l p e i s d
u| v p s j m g u a l r c o x b i n w d k q f h t e
v| n k h u r x v o f s i d p l e j b t g c w m a q
w| m j g s p v w n d t h e q k f l a u i b x o c r
x| o l i t q w x m e u g f r j d k c s h a v n b p
group is abelian: False
order of group: 24




## Завдання 2. 

__Знайдіть к-ть елементів кожного можливого порядку в групах $S_{100}$ та $A_{100}$.__

___Зауваження___: вивід буде великий, збережіть його в окремий текстовий файл

___Зауваження 2___: оцініть спершу к-ть елементів в групі, а потім ще раз подумайте чи варто тут писати повний перебір

-----

Нехай маємо елемент групи підстановок $g \in S_n$. Запишемо його як добуток незалежних циклів $g = c_1 c_2 \dots c_k$. Тоді порядок елемента записується як найменше спільне кратне довжин циклів: 

$$order(g) = lcm(len(c_1), len(c_2), \dots len(c_k))$$

Отже, порядок елемента залежить виключно від довжин циклів. Це рівносильно задачі знайти всі розбиття числа $n$ на суму довільної к-ті додатніх чисел, також знаної як [Integer Partitions](https://en.wikipedia.org/wiki/Integer_partition).

Заилишилось лише визначити скільки елементів у групі $S_n$ мають задані довжини незалежних циклів $(c_1, c_2, c_3, \dots c_k)$, i.e. мають заданий ___цикловий тип___.

In [56]:
el = SymmetricGroup(100).random_element()
el.cycle_type()

[44, 26, 11, 10, 4, 4, 1]

In [57]:
el

(1,22,92,31)(2,78,89,15,99,32,10,90,74,50,71,40,91,16,73,18,41,20,48,70,35,97,86,59,39,33,44,96,42,36,79,67,88,34,98,83,85,76,13,81,58,26,87,37)(3,55,45,25,51,28,8,62,53,30,6,49,46,21,19,94,77,12,61,82,4,43,9,60,56,17)(5,24,65,63,14,7,23,57,64,93)(11,29,47,100,54,38,52,95,69,75,68)(27,80,72,66)

In [149]:
factorials = [factorial(i) for i in range(100)]


def number(partition, n):
    """К-ть підстановок у групі S_n заданого циклового типу. 
    """
    res = 1
    for length in partition:
        # обираємо c_i чисел із тих, що лишились. Їх порядок важливий з точністю до зсува
        res *= binomial(n, length) * factorials[length - 1]

        # зменшуємо к-ть чисел 
        n -= length 
    
    # цикли однакової довжини можна міняти місцями, тому треба поділити на факторіали їх кількостей
    for amount in Counter(partition).values():
        res /= factorials[amount]
    return res

In [150]:
number([1, 2, 3, 4], 10)

151200

In [80]:
# перевірим брутфорсом правильність 
res = 0 
for el in tqdm(SymmetricGroup(10), total=int(factorial(10))): 
    if sorted(el.cycle_type()) == [1, 2, 3, 4]: 
        res += 1 
res

  0%|          | 0/3628800 [00:00<?, ?it/s]

151200

#### перевіримо, що розбиття числа дійсно швидше рахуються за класи спряженості

In [55]:
n = 30
G = SymmetricGroup(n)
t = time.time() 
G.conjugacy_classes_representatives()
print('elapsed:', time.time() - t)

t = time.time() 
list(Partitions(n))
print('elapsed:', time.time() - t)

elapsed: 18.672693014144897
elapsed: 0.011028051376342773


### Оцінка проблеми

In [87]:
# к-ть різних розбиттів, взято з https://math.stackexchange.com/questions/2675382/calculating-integer-partitions
# сама послідовність a(n) є в базі OEIS: https://oeis.org/A000041

def pentagonal_number(k):
    return int(k*(3*k-1) / 2)

def compute_partitions(goal):
    partitions = [1]
    for n in range(1,goal+1):
        partitions.append(0)
        for k in range(1,n+1):
            coeff = (-1)**(k+1)
            for t in [pentagonal_number(k), pentagonal_number(-k)]:
                if (n-t) >= 0:
                    partitions[n] = partitions[n] + coeff*partitions[n-t]
    return partitions

compute_partitions(100)[-10:]

[64112359,
 72533807,
 82010177,
 92669720,
 104651419,
 118114304,
 133230930,
 150198136,
 169229875,
 190569292]

#### Отже, нам треба буде перебрати 190млн різних розбиттів і для кожного порахувати к-ть підстановок з таким цикловим типом. Оцінимо скільки часу це займе.

In [373]:
import numpy as np 

res = []

for _ in range(100):
    el = Partitions(100).random_element()
    t = time.time() 
    number(partition=el, n=100)
    res.append(time.time() - t)

print(f'elapsed: {np.mean(res):.5f}s +- {np.std(res):.5f}')

mean_hours = (np.mean(res) * 190569292) / 3600
std_hours = np.std(res) * 190569292 / 3600
print(f'estimated time: {mean_hours:.2f}h +- {std_hours:.2f}h')

elapsed: 0.00053s +- 0.00029
estimated time: 27.85h +- 15.30h


#### Результат трохи невтішний. Прогнозується 30 годин на роботу алгоритму. Спробуємо оптимізувати функцію 

In [316]:
factors_cached = [0] + [list(factor(i)) for i in range(1, 101)]
primes = [0] + [el in Primes(101) for el in range(1, 101)]


def number_optimized(partition, n, factorize=False): 
    # дільники результуючого числа 
    res_factors = [0 for _ in range(n+1)]
    n_backup = n
    counter = [0 for _ in range(n+1)]
    for length in partition: 
        
        for i in range(1, n+1):    # * n!
            res_factors[i] += 1
        
        for i in range(1, n-length+1):   # / (n-k)!
            res_factors[i] -= 1
            
        res_factors[length] -= 1
        n -= length
        counter[length] += 1

    n = n_backup
    
    for amount in counter:
        if amount == 1 or amount == 0: continue 
        for i in range(1, amount+1):
            res_factors[i] -= 1

    if factorize:
        for i in range(2, n+1):
            if primes[i]: continue 
            if not res_factors[i]: continue
    
            i_amount = res_factors[i]
            for fact, f_amount in factors_cached[i]: 
                res_factors[fact] += f_amount * i_amount
            res_factors[i] -= i_amount
    res = 1
    for i in range(2, n+1): 
        res *= (i ** res_factors[i])
    return res, res_factors

In [317]:
el = Partitions(20).random_element()

number(el, 20)

2896311914496000

In [379]:
number([50, 50], 100)

18665243088788830536339847771253340098143193652876324293718592779043519998645983121788292795231303657250739584165444751650237042183372800000000000000000000

In [318]:
el

[7, 5, 3, 2, 2, 1]

In [201]:
!g++ cycletype.cpp &> /dev/null

In [202]:
!./a.out 1 2 3 4

0 -1 0 0 0 1 1 1 1 1 1 1 


In [166]:
!cat cycletype.cpp


#include <vector>
#include <iostream>


std::vector<int> factors(std::vector<int> &partition, int n){
    std::vector<int> res_factors(n+1, 0);
    std::vector<int> counter(n+1, 0);
    int n_backup = n;
    for (auto length : partition) {
        for (int i = 1; i < n+1; i++) res_factors[i]++;
        for (int i = 1; i < n - length + 1; i++) res_factors[i]--;
        res_factors[length]--;
        n -= length;

        counter[length]++;    // calculate Counter(partition)
    };

    for (auto amount : counter)
        for (int i = 1; i < amount; i++)
            res_factors[i]--;
    return res_factors;
}


int main(int argc, char *argv[]){
    std::vector<int> partitions;
    int n;
    for (int i = 1; i < argc; i++){
        int tmp = atoi(argv[i]);
        partitions.push_back(tmp);
        n += tmp;
    }
    auto res = factors(partitions, n);
    for (auto el : res){
        std::cout << el << ' ';
    }
    std::cout << std::endl;
}


In [377]:
import numpy as np 

res = []

for _ in range(100):
    el = Partitions(100).random_element()
    t = time.time() 
    number_optimized(partition=el, n=100)
    res.append(time.time() - t)

print(f'elapsed: {np.mean(res):.5f}s +- {np.std(res):.5f}')

mean_hours = (np.mean(res) * 190569292) / 3600
std_hours = np.std(res) * 190569292 / 3600
print(f'estimated time: {mean_hours:.2f}h +- {std_hours:.2f}h')

elapsed: 0.00022s +- 0.00011
estimated time: 11.88h +- 5.97h


In [384]:


i = 0

for el in Partitions(100):
    if i > 10:
        break

    print(number_optimized(el, 100, factorize=True))
    i += 1


print(-1)

(933262154439441526816992388562667004907159682643816214685929638952175999932299156089414639761565182862536979208272237582511852109168640000000000000000000000, [0, 1, 95, 48, 0, 22, 0, 16, 0, 0, 0, 9, 0, 7, 0, 0, 0, 5, 0, 5, 0, 0, 0, 4, 0, 0, 0, 0, 0, 3, 0, 3, 0, 0, 0, 0, 0, 2, 0, 0, 0, 2, 0, 2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0])
(942689044888324774562618574305724247380969376407895166349423877729470707002322379888297615920772911982360585058860846042941264756736000000000000000000000000, [0, 0, 97, 46, 0, 24, 0, 16, 0, 0, 0, 8, 0, 7, 0, 0, 0, 5, 0, 5, 0, 0, 0, 4, 0, 0, 0, 0, 0, 3, 0, 3, 0, 0, 0, 0, 0, 2, 0, 0, 0, 2, 0, 2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0])
(476154160428286493273975708450340308626101878899906231982617162

In [ ]:
def orders(n): 

    for part in Partition(10): 
        

In [21]:
orders = {}
for el in tqdm(G, total=int(G.order())): 
    ord = el.order() 
    if ord not in orders: 
        orders[ord] = 0 
    orders[ord] += 1 
orders

  0%|          | 0/3628800 [00:00<?, ?it/s]

{1: 1,
 10: 514080,
 2: 9495,
 5: 78624,
 9: 403200,
 21: 172800,
 4: 209160,
 12: 403200,
 6: 584640,
 8: 453600,
 3: 31040,
 7: 86400,
 20: 181440,
 30: 120960,
 14: 259200,
 15: 120960}

In [23]:
n = 100
length = 50
binomial(n, length)

100891344545564193334812497256

In [42]:
import time
t = time.time()
binomial(n , length) * factorial(length - 1)
print(time.time() - t)

0.00029277801513671875


In [45]:
list(Partitions(10))

[[10],
 [9, 1],
 [8, 2],
 [8, 1, 1],
 [7, 3],
 [7, 2, 1],
 [7, 1, 1, 1],
 [6, 4],
 [6, 3, 1],
 [6, 2, 2],
 [6, 2, 1, 1],
 [6, 1, 1, 1, 1],
 [5, 5],
 [5, 4, 1],
 [5, 3, 2],
 [5, 3, 1, 1],
 [5, 2, 2, 1],
 [5, 2, 1, 1, 1],
 [5, 1, 1, 1, 1, 1],
 [4, 4, 2],
 [4, 4, 1, 1],
 [4, 3, 3],
 [4, 3, 2, 1],
 [4, 3, 1, 1, 1],
 [4, 2, 2, 2],
 [4, 2, 2, 1, 1],
 [4, 2, 1, 1, 1, 1],
 [4, 1, 1, 1, 1, 1, 1],
 [3, 3, 3, 1],
 [3, 3, 2, 2],
 [3, 3, 2, 1, 1],
 [3, 3, 1, 1, 1, 1],
 [3, 2, 2, 2, 1],
 [3, 2, 2, 1, 1, 1],
 [3, 2, 1, 1, 1, 1, 1],
 [3, 1, 1, 1, 1, 1, 1, 1],
 [2, 2, 2, 2, 2],
 [2, 2, 2, 2, 1, 1],
 [2, 2, 2, 1, 1, 1, 1],
 [2, 2, 1, 1, 1, 1, 1, 1],
 [2, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]

## Завдання 3. 
___Для заданих натуральних $n, k$ ($1 <= n \le 1 000 000, 1 \le k \le n!$) визначте чи існує в групі $S_n$ елемент порядку $k$.___

----------

___Зауваження:___ зверніть увагу на межі, в яких задано $n, k$. 

In [ ]:
# ваш код тут